In [3]:
import tensorflow as tf
from psutil import virtual_memory
from huggingface_hub import notebook_login, InferenceClient
from langchain_core.prompts import ChatPromptTemplate
import torch

try:
  from langchain import HuggingFacePipeline
  from langchain import PromptTemplate, LLMChain
  from transformers import BitsAndBytesConfig
  from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
except:
  !pip install -q -U bitsandbytes accelerate langchain_community
  from langchain import HuggingFacePipeline
  from langchain import PromptTemplate, LLMChain
  from transformers import BitsAndBytesConfig
  from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


try:
  import g4f
  from g4f.client import Client
except:
  ! pip install -q -U g4f
  import g4f
  from g4f.client import Client


import getpass
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.2 MB/s eta 0:00:00


## Check RAM capacity

In [ ]:
# Check GPU
gpu_info = tf.config.list_physical_devices('GPU')
print(f"GPU Info: {gpu_info}")

# Check RAM
ram_info = virtual_memory()
print(f"Total RAM: {ram_info.total / (1024**3)} GB")

In [ ]:
! nvidia-smi

Mon Feb  3 08:55:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Serverless Inference API of huggingface

Serverless Inference API of huggingface: allows you to run LLMs without needing to load them on your local machine or Colab. Instead, you send a request to Hugging Face's cloud service, which runs the model and returns the output.

The Serverless Inference API offers a fast and free way to explore thousands of models for a variety of tasks. Whether you’re prototyping a new application or experimenting with ML capabilities, this API gives you instant access to high-performing models across multiple domains


Main Features
- Leverage over 800,000+ models from different open-source libraries (transformers, sentence transformers, adapter transformers, diffusers, timm, etc.).
- Use models for a variety of tasks, including text generation, image generation, document embeddings, NER, summarization, image classification, and more.
- **Accelerate your prototyping by using GPU-powered models.
Run very large models that are challenging to deploy in production**.  (I need this 🙂)
- Production-grade platform without the hassle: built-in automatic scaling, load balancing and caching.



You can use **Serverless Inference API** of huggingface
before you can use it, create an `API_KEY` from this [link](https://huggingface.co/settings/tokens/new?globalPermissions=inference.serverless.write&tokenType=fineGrained)

Read more:

https://huggingface.co/docs/api-inference/en/index

https://huggingface.co/docs/api-inference/en/tasks/chat-completion

In [4]:
# Log in to Hugging Face so you can use Hugging Face LLM models. To do this, you need to create an Access Token on Hugging Face.
notebook_login()

In [5]:
os.environ["HUGGINGFACE_SERVERLESS_API_KEY"] = getpass.getpass()

··········


In [6]:
client = InferenceClient(api_key=os.getenv("HUGGINGFACE_SERVERLESS_API_KEY"))

In [8]:
messages = [
	{
		"role": "user",
		"content": "Give me an Iranian food recipe"
    	}
]

stream = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-32B",
		messages=messages,
		max_tokens=2500,
		stream=True
)

for chunk in stream:
    print(chunk.choices[0].delta.content, end="")

<think>
Okay, so I want to make this Iranian dish called Mirza Ghasemi. I've heard it's a traditional breakfast or snack, so that sounds manageable for someone like me who's just starting out in the kitchen. Let's see, the ingredients needed are eggplants, eggs, tomatoes, onions, garlic, some herbs like parsley and dill, and some spices like turmeric, cumin, and red pepper. Oh, and it's served with bread, which is easy enough.

First, I think I need to prepare the eggplant. The recipe says to wash and slice them into about 1-inch thick rounds. I'm wondering if there's a specific type of eggplant that works best here, or if any will do. I'll probably use whatever's available at the store.

Next step is grilling the eggplant. I don't have an outdoor grill, so maybe I can use a pan on the stove. The recipe mentions placing them under a preheated broiler or grilling until the slices are charred. If using a pan, I should maybe put some oil in it and cook the eggplant until they’re soft and 

## g4f

g4f: is a Python library that allows you to access ChatGPT, DeepSeek-r1 and other LLMs for free by leveraging various reverse-engineered API endpoints from different providers. You can find supported LLMs from [here](https://github.com/xtekky/gpt4free/blob/main/docs/providers-and-models.md).

In [17]:
g4f.debug.logging = True # enable logging
g4f.check_version = False # Disable automatic version checking
print(g4f.version) # check version
print(g4f.Provider.Ails.params)  # supported args

# Automatic selection of provider

# streamed completion
response = g4f.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Give me an Iranian food recipe"}],
    stream=True,
)

for message in response:
    print(message, flush=True, end='')

# # normal response
# response = g4f.ChatCompletion.create(
#     model=g4f.models.gpt_4,
#     messages=[{"role": "user", "content": "Hello"}],
# )  # alternative model setting

# print(response)

<module 'g4f.version' from '/usr/local/lib/python3.11/dist-packages/g4f/version.py'>
g4f.Provider.Ails supports: (
    model: str,
    messages: Messages,
    stream: bool,
    proxy: str = None,
)
Using RetryProvider provider and gpt-4 model
Using OpenaiChat provider
OpenaiChat: NoValidHarFileError: har_and_cookies dir is not readable
har_and_cookies dir is not readableUsing Yqcloud provider
Here’s a classic Iranian recipe for **Fesenjan**, a delicious Persian stew made with chicken, pomegranate, and walnuts:

### **Fesenjan (Pomegranate and Walnut Stew)**

#### **Ingredients:**
- 2 tbsp olive oil or vegetable oil
- 2 onions, finely chopped
- 2 chicken breasts or thighs, cut into pieces (or duck, if preferred)
- 1 ½ cups ground walnuts
- 1 ½ cups pomegranate molasses (or fresh pomegranate juice with sugar if molasses is unavailable)
- 1 tsp ground turmeric
- 1 tsp ground cinnamon (optional)
- 1 tsp ground saffron (optional)
- Salt and pepper to taste
- 1 to 2 cups chicken broth (or wa

In [15]:
g4f.ChatCompletion.create?

Image generation with gpt-4

In [ ]:
client = Client()
response = client.images.generate(
    model="flux",
    prompt="A cat on milad tower in Tehran",
    response_format="url"
    # Add any other necessary parameters
)

image_url = response.data[0].url
print(f"Generated image URL: {image_url}")

## Quantization

Quantization is a technique that reduces the precision of a model’s numerical parameters (e.g., from 32-bit floating point to 8-bit integers), significantly lowering memory usage and computational requirements. This makes it possible to run large LLMs on resource-limited environments like free Colab, with minimal performance loss. Common quantization methods include int8, int4, and GPTQ (grouped quantization), often used in library like BitsAndBytes.

### Mistral-7B-Instruct (Quantized)

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

first we need to get a grant to access the `/Mistral-7B-Instruct-v0.1`. go to repo of the model in huggingface and request for grant.


In [ ]:
mistralai_model_4bit = AutoModelForCausalLM.from_pretrained( "mistralai/Mistral-7B-Instruct-v0.1", device_map="auto",quantization_config=quantization_config, )
mistralai_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
mistralai_pipeline_inst = pipeline(
        "text-generation",
        model=mistralai_model_4bit,
        tokenizer=mistralai_tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2500,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=mistralai_tokenizer.eos_token_id,
        pad_token_id=mistralai_tokenizer.eos_token_id,
)

mistralai_llm = HuggingFacePipeline(pipeline=mistralai_pipeline_inst)

Device set to use cuda:0
<ipython-input-8-2583d39fa3e7>:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  mistralai_llm = HuggingFacePipeline(pipeline=mistralai_pipeline_inst)


In [ ]:
template = """You are a professional chef assistant.
Your task is to provide detailed and accurate recipes.
Each recipe should include ingredients, step-by-step instructions, and any relevant cooking tips.

{question}"""

def generate_response(question, llm, template):
  prompt = PromptTemplate(template=template, input_variables=["question"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"question":question})
  return " ".join(response.split())

In [ ]:
generate_response("Give me an Iranian food recipe", mistralai_llm, template)

<ipython-input-9-94271b465e86>:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)
<ipython-input-9-94271b465e86>:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run({"question":question})
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'You are a professional chef assistant. Your task is to provide detailed and accurate recipes. Each recipe should include ingredients, step-by-step instructions, and any relevant cooking tips. Give me an Iranian food recipe. Iranian Kookook Ingredients: - 2 cups of rice flour - 1 cup of wheat flour - 1 tsp of salt - 1 tsp of turmeric - 1 tsp of paprika - 2 eggs - 2 cups of water - 2 tbsp of butter - 2 tbsp of oil - 1 cup of diced onion - 3 cloves of garlic, minced - 1 cup of diced bell pepper - 1 cup of diced tomato - 1/2 cup of chopped parsley - 1/2 cup of chopped cilantro - 1/4 cup of chopped mint - 1/4 cup of chopped cherry tomatoes - 1/4 cup of chopped green onions - 3 tbsp of red wine vinegar - Salt and pepper to taste Instructions: 1. In a large mixing bowl, sift flour with salt, turmeric, and paprika, then mix with rice flour. 2. Beat the eggs, pour into a bowl and mix with flour mixture. Gradually add flour to egg mixture, stirring constantly until well mixed. 3. Add water to t

### google/gemma-7b-it (Quantized)

first we need to get a grant to access the `gemma-7b-it`. go to repo of the model in huggingface and request for grant.

In [ ]:
gemma7b_model_4bit = AutoModelForCausalLM.from_pretrained( "google/gemma-7b-it", device_map="auto",quantization_config=quantization_config, )
gemma7b_tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
gemma7b_pipeline_inst = pipeline(
        "text-generation",
        model=gemma7b_model_4bit,
        tokenizer=gemma7b_tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2500,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=gemma7b_tokenizer.eos_token_id,
        pad_token_id=gemma7b_tokenizer.eos_token_id,
)

gemma7b_llm = HuggingFacePipeline(pipeline=gemma7b_pipeline_inst)

Device set to use cuda:0
<ipython-input-6-567147dbdd45>:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  gemma7b_llm = HuggingFacePipeline(pipeline=gemma7b_pipeline_inst)


In [ ]:
generate_response("Give me an Iranian food recipe", gemma7b_llm, template)

<ipython-input-4-94271b465e86>:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)
<ipython-input-4-94271b465e86>:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run({"question":question})
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'You are a professional chef assistant. Your task is to provide detailed and accurate recipes. Each recipe should include ingredients, step-by-step instructions, and any relevant cooking tips. Give me an Iranian food recipe for a delicious Lamb Shank with Saffron Rice. **Lamb Shank with Saffron Rice** **Ingredients:** * Lamb shanks (2) * Rice (2 cups) * Chicken broth (4 cups) * Saffron seasoning (1 tsp) * Garlic powder (1 tbsp) * Cumin seeds (1 tsp) * Dried mint (1 tbsp) * Carrots (2) * Parsnips (2) * Onions (2) * Tomatoes (2) * Tomato paste (1 cup) * Butter (1/2 cup) * Olive oil (1/4 cup) * Fresh herbs (mint, parsley) **Instructions:** 1. Preheat oven to 350 degrees F (175 degrees C). 2. In a large pot, bring the chicken broth, saffron seasoning, garlic powder, cumin seeds, and dried mint to a boil. Reduce heat to low and let simmer for 2 hours, or until the lamb shanks are tender. 3. Remove the lamb shanks from the broth and set aside. 4. In a separate pot, bring the rice and water t